In [1]:
#Import Libraries Needed
import pandas as pd
import numpy as np
import geopandas as gpd
import panel as pn
pn.extension('plotly',comms='ipywidgets')
import param
import bokeh
import plotly
from bokeh.resources import INLINE
import hvplot.pandas
import plotly.express as px
from urllib.request import urlopen
import plotly.graph_objects as go
import requests
from PIL import Image, ImageOps
import json

In [2]:
css = '''
.bk.panel-widget-box {
  background: #f0f0f0;
  border-radius: 5px;
  border: 1px black solid;
  font-size: 24px;
}

.title-bar {
  display: flex;
  justify-content: space-around;
  align-items: center;
  flex-wrap: wrap;
}

.title-html-pane {
  color: white;
  font-size: 20px;
  background-color: #6A5638;
  text-align: center;
  padding: 4px;
  font-family: Times New Roman;
  max-width: 800px;
  width: 100%;
  outline-style: solid;
  outline-color: #30231D;
  outline-width: thick;
  display: flex;
}

.image-box {
  display: flex;
  justify-content: center;
  align-items: center;
  background-color: white;
  padding: 4px;
}


}
'''

pn.extension(raw_css=[css])

In [3]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
target_states = ['48']
counties['features'] = [f for f in counties['features'] if f['properties']['STATE'] in target_states]
address = ("Data/District_Loss/district18-22.csv")
dis_df = pd.read_csv(address)

#Create a new dataframe with columns based off the differences of 2021 National Center on Education Statistics and
#2019 National Center on Education Statistics, divided by the 2019 data.
dis_copy = dis_df.copy()
countynames = dis_df['County Name'].unique()
countynames.sort()
fips = dis_df['County #'].unique()
fips.sort()
dis_copy = dis_copy.groupby('County Name').mean()
dis_copy = dis_copy.drop(['District #'], axis = 1)
dis_copy['County #'] = fips
dis_copy['County Name'] = countynames
dis_copy= dis_copy.reset_index(drop=True)

dis_copy['Full-Time Equivalent(FTE) Teachers Difference (2021-2019)'] = (dis_copy['FTE Teachers 2020-2021']-dis_copy['FTE Teachers 2018-2019'])/dis_copy['FTE Teachers 2018-2019']
dis_copy['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2019)'] = (dis_copy['Total Staff 2020-2021']-dis_copy['Total Staff 2018-2019'])/dis_copy['Total Staff 2018-2019']
dis_copy['Total Number of Students Difference (2021-2019)'] = (dis_copy['Total Students 2020-2021']-dis_copy['Total Students 2018-2019'])/dis_copy['Total Students 2018-2019']

dis_copy['Full-Time Equivalent(FTE) Teachers Difference (2021-2022)'] = (dis_copy['FTE Teachers 2021-2022']-dis_copy['FTE Teachers 2020-2021'])/dis_copy['FTE Teachers 2020-2021']
dis_copy['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2022)'] = (dis_copy['Total Staff 2021-2022']-dis_copy['Total Staff 2020-2021'])/dis_copy['Total Staff 2020-2021']
dis_copy['Total Number of Students Difference (2021-2022)'] = (dis_copy['Total Students 2021-2022']-dis_copy['Total Students 2020-2021'])/dis_copy['Total Students 2020-2021']

dis_copy['Full-Time Equivalent(FTE) Teachers Difference (2019-2022)'] = (dis_copy['FTE Teachers 2021-2022']-dis_copy['FTE Teachers 2018-2019'])/dis_copy['FTE Teachers 2018-2019']
dis_copy['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2022)'] = (dis_copy['Total Staff 2021-2022']-dis_copy['Total Staff 2018-2019'])/dis_copy['Total Staff 2018-2019']
dis_copy['Total Number of Students Difference (2019-2022)'] = (dis_copy['Total Students 2021-2022']-dis_copy['Total Students 2018-2019'])/dis_copy['Total Students 2018-2019']

C:\Users\Leaf\AppData\Local\Temp\ipykernel_23488\3141300911.py:15: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dis_copy = dis_copy.groupby('County Name').mean()


In [4]:
dis_copy = dis_copy.round(2)

In [5]:
#Import dataframes
address = ("Data/District_Loss/txcountyemploy.xlsx")
employ_df = pd.read_excel(address)

#Create a new dataframes with columns based on the differences of the 2021 Bureau of Labor Statistics Data and 
#2019 Bureau of Labor Statistics Data, divided by the 2019 data.
employ_df['NAME'] = employ_df['NAME'].str.replace(", TX","")
employ_df['YEAR']=employ_df['YEAR'].values.astype(str)
employ_df['YEAR'] = employ_df['YEAR'].str.replace("'","")
df_19 = employ_df[employ_df['YEAR'] == '2019']
df_20 = employ_df[employ_df['YEAR'] == '2020']
df_21 = employ_df[employ_df['YEAR'] == '2021']
df_20 =df_20.reset_index()
df_21=df_21.reset_index()
diff_df = pd.DataFrame()
diff_df = df_19[['NAME','FIPS']]
diff_df['Civilian Labor Force Difference (2021-2019)'] = (df_21['LABOR_FORCE']-df_19['LABOR_FORCE'])/df_19['LABOR_FORCE']
diff_df['Employed Persons Difference (2021-2019)']= (df_21['EMPLOYED']-df_19['EMPLOYED'])/df_19['EMPLOYED']
diff_df['Umemployed Persons Difference (2021-2019)']= (df_21['UNEMPLOYED']-df_19['UNEMPLOYED'])/df_19['UNEMPLOYED']
diff_df['Unemployment Rate Difference (2021-2019)']= (df_21['UNEMPLOYMENT_PCT']-df_19['UNEMPLOYMENT_PCT'])/df_19['UNEMPLOYMENT_PCT']

c:\Users\Leaf\anaconda3\envs\EducationDashboard\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Leaf\AppData\Local\Temp\ipykernel_23488\1615083697.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_df['Civilian Labor Force Difference (2021-2019)'] = (df_21['LABOR_FORCE']-df_19['LABOR_FORCE'])/df_19['LABOR_FORCE']
C:\Users\Leaf\AppData\Local\Temp\ipykernel_23488\1615083697.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [6]:
diff_df = diff_df.round(2)
diff_df.rename(columns={'NAME':'County Name','FIPS':'County #'},inplace=True)

In [7]:
#Import dataframes
address = ("Data/District_Loss/DATA_Texas_District_v1.csv")
disloss_df = pd.read_csv(address)
address = ("Data/District_Loss/district18-22.csv")
dis_df = pd.read_csv(address)

#Creates a dataframe based on the Texas Education Agency data based on the differences of
#2021 and 2019 data, divided by 2019 data.
disloss_df['County #'] = disloss_df['County #'].astype(int)
countynames = dis_df['County Name'].unique()
countynames.sort()
fips = dis_df['County #'].unique()
fips.sort()

discopy_df = pd.DataFrame()
discopy_df['County #'] = disloss_df['County #']
discopy_df['Average Score Difference(%) in Math (2021-2019)'] = (disloss_df['AVG_Math_2021']-disloss_df['AVG_Math_2019'])/disloss_df['AVG_Math_2019']
discopy_df['Average Score Difference(%) in Reading (2021-2019)'] = (disloss_df['AVG_Reading_2021']-disloss_df['AVG_Reading_2019'])/disloss_df['AVG_Reading_2019']
discopy_df['Average Score Difference(%) in Math (2022-2021)'] = (disloss_df['AVG_Math_2022']-disloss_df['AVG_Math_2021'])/disloss_df['AVG_Math_2021']
discopy_df['Average Score Difference(%) in Reading (2022-2021)'] = (disloss_df['AVG_Reading_2022']-disloss_df['AVG_Reading_2021'])/disloss_df['AVG_Reading_2021']
discopy_df['Average Score Difference(%) in Math (2022-2019)'] = (disloss_df['AVG_Math_2022']-disloss_df['AVG_Math_2019'])/disloss_df['AVG_Math_2019']
discopy_df['Average Score Difference(%) in Reading (2022-2019)'] = (disloss_df['AVG_Reading_2022']-disloss_df['AVG_Reading_2019'])/disloss_df['AVG_Reading_2019']

#Discrete Version
#discopy_df['Average Score Difference(%) in Math (2021-2019)'] = (disloss_df['AVG_Math_2021']-disloss_df['AVG_Math_2019'])
#discopy_df['Average Score Difference(%) in Reading (2021-2019)'] = (disloss_df['AVG_Reading_2021']-disloss_df['AVG_Reading_2019'])
#discopy_df['Average Score Difference(%) in Math (2022-2021)'] = (disloss_df['AVG_Math_2022']-disloss_df['AVG_Math_2021'])
#discopy_df['Average Score Difference(%) in Reading (2022-2021)'] = (disloss_df['AVG_Reading_2022']-disloss_df['AVG_Reading_2021'])
#discopy_df['Average Score Difference(%) in Math (2022-2019)'] = (disloss_df['AVG_Math_2022']-disloss_df['AVG_Math_2019'])
#discopy_df['Average Score Difference(%) in Reading (2022-2019)'] = (disloss_df['AVG_Reading_2022']-disloss_df['AVG_Reading_2019'])

discopy_df = discopy_df.groupby('County #').mean()
discopy_df['County #'] = fips
discopy_df['County Name'] = countynames
discopy_df = discopy_df.reset_index(drop=True)

In [8]:
discopy_df = discopy_df.round(2)

In [9]:
discopy_df.drop(columns='County Name',inplace=True)
dis_copy.drop(columns='County Name',inplace=True)
comp_df = discopy_df.merge(diff_df,on='County #')
comp_df = comp_df.merge(dis_copy,on='County #')

In [10]:
comp_df.columns

Index(['Average Score Difference(%) in Math (2021-2019)',
       'Average Score Difference(%) in Reading (2021-2019)',
       'Average Score Difference(%) in Math (2022-2021)',
       'Average Score Difference(%) in Reading (2022-2021)',
       'Average Score Difference(%) in Math (2022-2019)',
       'Average Score Difference(%) in Reading (2022-2019)', 'County #',
       'County Name', 'Civilian Labor Force Difference (2021-2019)',
       'Employed Persons Difference (2021-2019)',
       'Umemployed Persons Difference (2021-2019)',
       'Unemployment Rate Difference (2021-2019)', 'FTE Teachers 2018-2019',
       'Total Staff 2018-2019', 'Total Students 2018-2019',
       'FTE Teachers 2020-2021', 'Total Staff 2020-2021',
       'Total Students 2020-2021', 'FTE Teachers 2021-2022',
       'Total Staff 2021-2022', 'Total Students 2021-2022',
       'Full-Time Equivalent(FTE) Teachers Difference (2021-2019)',
       'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (202

In [11]:
comp_df.rename(columns={'Full-Time Equivalent(FTE) Teachers Difference (2021-2019)':'Number of Teachers (2019-2021)',
                          'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2019)':'Number of Overall Staff (2019-2021)',
                          'Total Number of Students Difference (2021-2019)': 'Number of Students (2019-2021)',
                          'Civilian Labor Force Difference (2021-2019)':'Labor Force (2019-2021)',
    
                          'Average Score Difference(%) in Math (2021-2019)': 'STAAR Score in Math (2019-2021)',
                          'Average Score Difference(%) in Reading (2021-2019)': 'STAAR Score in Reading (2019-2021)',
                          'Average Score Difference(%) in Math (2022-2021)': 'STAAR Score in Math (2021-2022)',
                          'Average Score Difference(%) in Reading (2022-2021)': 'STAAR Score in Reading (2021-2022)',
                          'Average Score Difference(%) in Math (2022-2019)': 'STAAR Score in Math (2019-2022)',
                          'Average Score Difference(%) in Reading (2022-2019)': 'STAAR Score in Reading (2019-2022)',
                        
                          'Full-Time Equivalent(FTE) Teachers Difference (2021-2022)':'Number of Teachers (2021-2022)',
                          'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2022)': 'Number of Overall Staff (2021-2022)',
                          'Total Number of Students Difference (2021-2022)': 'Number of Students (2021-2022)',
                        
                          'Full-Time Equivalent(FTE) Teachers Difference (2019-2022)':'Number of Teachers (2019-2022)',
                          'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2022)': 'Number of Overall Staff (2019-2022)',
                          'Total Number of Students Difference (2019-2022)': 'Number of Students (2019-2022)'},
                          inplace=True)

In [12]:
comp_df['Number of Teachers (2019-2021)'] = comp_df['Number of Teachers (2019-2021)']*100
comp_df['Number of Overall Staff (2019-2021)'] = comp_df['Number of Overall Staff (2019-2021)']*100
comp_df['Number of Students (2019-2021)'] = comp_df['Number of Students (2019-2021)']*100

comp_df['Number of Teachers (2021-2022)'] = comp_df['Number of Teachers (2021-2022)']*100
comp_df['Number of Overall Staff (2021-2022)'] = comp_df['Number of Overall Staff (2021-2022)']*100
comp_df['Number of Students (2021-2022)'] = comp_df['Number of Students (2021-2022)']*100

comp_df['Number of Teachers (2019-2022)'] = comp_df['Number of Teachers (2019-2022)']*100
comp_df['Number of Overall Staff (2019-2022)'] = comp_df['Number of Overall Staff (2019-2022)']*100
comp_df['Number of Students (2019-2022)'] = comp_df['Number of Students (2019-2022)']*100

# TEMPORARILY COMMENT OUT, WE DONT NEED TO MULTIPLY BY 100 AS WE ARE NOT USING PERCENTAGES ANYMORE
comp_df['STAAR Score in Math (2019-2021)'] = comp_df['STAAR Score in Math (2019-2021)']*100
comp_df['STAAR Score in Reading (2019-2021)'] = comp_df['STAAR Score in Reading (2019-2021)']*100
comp_df['STAAR Score in Math (2021-2022)'] = comp_df['STAAR Score in Math (2021-2022)']*100
comp_df['STAAR Score in Reading (2021-2022)'] = comp_df['STAAR Score in Reading (2021-2022)']*100
comp_df['STAAR Score in Math (2019-2022)'] = comp_df['STAAR Score in Math (2019-2022)']*100
comp_df['STAAR Score in Reading (2019-2022)'] = comp_df['STAAR Score in Reading (2019-2022)']*100

comp_df['Labor Force (2019-2021)'] = comp_df['Labor Force (2019-2021)']*100
comp_df['County Name'] = comp_df['County Name'].str[:-7]

In [13]:
comp_df.dtypes

STAAR Score in Math (2019-2021)              float64
STAAR Score in Reading (2019-2021)           float64
STAAR Score in Math (2021-2022)              float64
STAAR Score in Reading (2021-2022)           float64
STAAR Score in Math (2019-2022)              float64
STAAR Score in Reading (2019-2022)           float64
County #                                       int64
County Name                                   object
Labor Force (2019-2021)                      float64
Employed Persons Difference (2021-2019)      float64
Umemployed Persons Difference (2021-2019)    float64
Unemployment Rate Difference (2021-2019)     float64
FTE Teachers 2018-2019                       float64
Total Staff 2018-2019                        float64
Total Students 2018-2019                     float64
FTE Teachers 2020-2021                       float64
Total Staff 2020-2021                        float64
Total Students 2020-2021                     float64
FTE Teachers 2021-2022                       f

# Texas County Level Map

In [14]:
comp_df['STAAR Score in Math (2021-2022)'] = comp_df['STAAR Score in Math (2021-2022)'].astype(int)
comp_df['STAAR Score in Reading (2021-2022)'] = comp_df['STAAR Score in Reading (2021-2022)'].astype(int)

comp_df['STAAR Score in Math (2019-2022)'] = comp_df['STAAR Score in Math (2019-2022)'].astype(int)
comp_df['STAAR Score in Reading (2019-2022)'] = comp_df['STAAR Score in Reading (2019-2022)'].astype(int)

comp_df['STAAR Score in Math (2019-2021)'] = comp_df['STAAR Score in Math (2019-2021)'].astype(int)
comp_df['STAAR Score in Reading (2019-2021)'] = comp_df['STAAR Score in Reading (2019-2021)'].astype(int)
comp_df['Labor Force (2019-2021)'] = comp_df['Labor Force (2019-2021)'].astype(int)


comp_df['Number of Teachers (2021-2022)'] = comp_df['Number of Teachers (2021-2022)'].astype(int)
comp_df['Number of Overall Staff (2021-2022)'] = comp_df['Number of Overall Staff (2021-2022)'].astype(int)
comp_df['Number of Students (2021-2022)'] = comp_df['Number of Students (2021-2022)'].astype(int)

comp_df['Number of Teachers (2019-2022)'] = comp_df['Number of Teachers (2019-2022)'].astype(int)
comp_df['Number of Overall Staff (2019-2022)'] = comp_df['Number of Overall Staff (2019-2022)'].astype(int)
comp_df['Number of Students (2019-2022)'] = comp_df['Number of Students (2019-2022)'].astype(int)

comp_df['Number of Teachers (2019-2021)'] = comp_df['Number of Teachers (2019-2021)'].astype(int)
comp_df['Number of Overall Staff (2019-2021)'] = comp_df['Number of Overall Staff (2019-2021)'].astype(int)
comp_df['Number of Students (2019-2021)'] = comp_df['Number of Students (2019-2021)'].astype(int)

In [15]:
comp_df.to_csv('Data/CountyLevelstats.csv')

In [16]:
df1921 = comp_df[['County Name','County #','Number of Teachers (2019-2021)',
                  'Number of Overall Staff (2019-2021)',
                  'Number of Students (2019-2021)',
                  'Labor Force (2019-2021)',
                  'STAAR Score in Math (2019-2021)',
                  'STAAR Score in Reading (2019-2021)']]

df2122 = comp_df[['County Name','County #','Number of Teachers (2021-2022)',
                  'Number of Overall Staff (2021-2022)',
                  'Number of Students (2021-2022)',
                  'STAAR Score in Math (2021-2022)',
                  'STAAR Score in Reading (2021-2022)']]

df1922 = comp_df[['County Name','County #','Number of Teachers (2019-2022)',
                  'Number of Overall Staff (2019-2022)',
                  'Number of Students (2019-2022)',
                  'STAAR Score in Math (2019-2022)',
                  'STAAR Score in Reading (2019-2022)']]

In [17]:
df1921.rename(columns = {'Number of Teachers (2019-2021)':'Number of Teachers',
                           'Number of Overall Staff (2019-2021)':'Number of Staff',
                           'Number of Students (2019-2021)':'Number of Students',
                           'STAAR Score in Math (2019-2021)':'STAAR Score in Math',
                           'STAAR Score in Reading (2019-2021)':'STAAR Score in Reading'
                           }, inplace=True)

df2122.rename(columns = {'Number of Teachers (2021-2022)':'Number of Teachers',
                           'Number of Overall Staff (2021-2022)':'Number of Staff',
                           'Number of Students (2021-2022)':'Number of Students',
                           'STAAR Score in Math (2021-2022)':'STAAR Score in Math',
                           'STAAR Score in Reading (2021-2022)':'STAAR Score in Reading',
                           }, inplace=True)

df1922.rename(columns = {'Number of Teachers (2019-2022)':'Number of Teachers',
                           'Number of Overall Staff (2019-2022)':'Number of Staff',
                           'Number of Students (2019-2022)':'Number of Students',
                           'STAAR Score in Math (2019-2022)':'STAAR Score in Math',
                           'STAAR Score in Reading (2019-2022)':'STAAR Score in Reading',
                           }, inplace=True)

C:\Users\Leaf\AppData\Local\Temp\ipykernel_23488\3827061595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1921.rename(columns = {'Number of Teachers (2019-2021)':'Number of Teachers',
C:\Users\Leaf\AppData\Local\Temp\ipykernel_23488\3827061595.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2122.rename(columns = {'Number of Teachers (2021-2022)':'Number of Teachers',
C:\Users\Leaf\AppData\Local\Temp\ipykernel_23488\3827061595.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [18]:
#Creates a county level map of Texas based on the Texas Education Agency Data
input_list = ['STAAR Score in Math',
              'STAAR Score in Reading',
              'Number of Students',
              'Number of Teachers',
              'Number of Staff']
# Labor Force Taken out for now 2019-2021
year_list = ['2019-2021','2021-2022','2019-2022']
class MapDashboard(param.Parameterized):
    _  = param.Selector(input_list,default=input_list[0])
    __ = param.Selector(default=year_list[0],objects=year_list)
    #@param.depends('_')
    def get_map(self):
        if self.__ == '2019-2021':
            data = df1921[['County Name','County #',self._]]
        elif self.__ == '2021-2022':
            data = df2122[['County Name','County #',self._]]
        elif self.__ == '2019-2022':
            data = df1922[['County Name','County #',self._]]
        fig = px.choropleth_mapbox(data, geojson=counties, locations='County #', color= self._,
        color_continuous_scale='balance_r',
        range_color=(-10,10),
        center = {'lat':31.9686, 'lon': -99.9018},
        zoom = 4.65,
        height=585, width=525,
        mapbox_style = "white-bg",       
        custom_data = [data['County Name'],self._]
        )
        hovertemp = ''
        hovertemp = '%{customdata[0]} County<br>'                   
        hovertemp += '%{customdata[1]}% change in ' + self._
        fig.update_traces(hovertemplate=hovertemp)
        fig.update_geos(fitbounds='locations')
        fig.update_layout(hoverlabel=dict(bgcolor='black',font_size=24,font_family="Times New Roman"),
        font=dict(family='times new roman',
        size=24,color='black'),                
        margin={"r":10,"t":0,"l":10,"b":0})
        fig.update_layout(coloraxis_colorbar=dict(tickvals=[-8,-4,0,4,8],ticktext=['-10','-5','0','5','10'])),
        fig.update_coloraxes(cmid=0,colorbar_orientation='h',colorbar_y=-.148,colorbar_title_font_family='Times New Roman',
                             colorbar_title_font_size=20,colorbar_title_side='bottom', colorbar_thickness=4)
        return fig
    def get_description(self):
        descriptions = {
            "2019-2021": {
                "STAAR Score in Math": "The percent difference in <b>Math STAAR scores</b> between <b>2019 and 2021</b> for a district.",
                "STAAR Score in Reading": "The percent difference in <b>Reading STAAR scores</b> between <b>2019 and 2021</b> for a district.",
                "Number of Teachers": "The percent difference in <b>Full-Time Teachers</b> between <b>2019 and 2021</b> for a district.",
                "Number of Staff": "The percent difference in <b>staff</b> between <b>2019 and 2021</b> for a district.",
                "Number of Students": "The percent difference in <b>students</b> between <b>2019 and 2021</b> for a district."
            },
            "2021-2022": {
                "STAAR Score in Math": "The percent difference in <b>Math STAAR scores</b> between <b>2021 and 2022</b> for a district.",
                "STAAR Score in Reading": "The percent difference in <b>Reading STAAR scores</b> between <b>2021 and 2022</b> for a district.",
                "Number of Teachers": "The percent difference in <b>Full-Time Teachers</b> between <b>2021 and 2022</b> for a district.",
                "Number of Staff": "The percent difference in <b>staff</b> between <b>2021 and 2022</b> for a district.",
                "Number of Students": "The percent difference in <b>students</b> between <b>2021 and 2022</b> for a district."
            },
            "2019-2022": {
                "STAAR Score in Math": "The percent difference in <b>Math STAAR scores</b> between <b>2019 and 2022</b> for a district.",
                "STAAR Score in Reading": "The percent difference in <b>Reading STAAR scores</b> between <b>2019 and 2022</b> for a district.",
                "Number of Teachers": "The percent difference in <b>Full-Time Teachers</b> between <b>2019 and 2022</b> for a district.",
                "Number of Staff": "The percent difference in <b>staff</b> between <b>2019 and 2022</b> for a district.",
                "Number of Students": "The percent difference in <b>students</b> between <b>2019 and 2022</b> for a district."
            },
        }

        if self.__ in descriptions:
            description_text = descriptions[self.__].get(self._)
            if description_text:
                return pn.pane.HTML(description_text, 
                                    styles={'color': 'black', 'font-size': '20px', 'width': '125%',
                                            'text-align': 'center', 'max-width': '500px', 'word-wrap': 'break-word',
                                            'margin-left': '147px', 'font-family': 'Times New Roman'})
        return None

    def panel(self):
        return pn.Row(
            pn.Column(
            pn.pane.HTML("""<h1>Select A Variable To View At The County Level</h1>""",
                              styles={'color':'black','font-size':'14px','width':'100%',
                                     'text-align':'center',
                                     'font-family':'Times New Roman'}),
            pn.Param(self,width=550,styles={'background':'#FFFFFF','margin-left':'225px'},
            widgets={'__':{'widget_type':pn.widgets.RadioButtonGroup}}),
            pn.Spacer(height=50),
            pn.pane.HTML("""<h1>What does this variable show us?</h1>""",
                            styles={'color':'black','font-size':'14px','width':'100%',
                                  'text-align':'center',
                                  'font-family':'Times New Roman'}),
            pn.Spacer(height= 5),
            self.get_description
            ),
            pn.Spacer(width=150),
            self.get_map
        )
dashboard = MapDashboard(name='')
map_component1 = dashboard.panel()
map_component1.save('CountyLevel_Tab',resources=INLINE)

# Heatmap & Histogram

In [19]:
#Import dataframe
address = ("Data/District_Loss/DATA_Texas_District_v1.csv")
loss_df = pd.read_csv(address)
loss_df = loss_df.round(2)

In [20]:
loss_df['% LOSS Math 2019 to 2022'] = loss_df['% LOSS Math 2019 to 2022']*100
loss_df['% LOSS Reading 2019 to 2022'] = loss_df['% LOSS Reading 2019 to 2022']*100
loss_df['% LOSS Math 2021 to 2022'] = loss_df['% LOSS Math 2021 to 2022']*100
loss_df['% LOSS Reading 2021 to 2022'] = loss_df['% LOSS Reading 2021 to 2022']*100
loss_df['% LOSS Math 2019 to 2021'] = loss_df['% LOSS Math 2019 to 2021']*100
loss_df['% LOSS Reading 2019 to 2021'] = loss_df['% LOSS Reading 2019 to 2021']*100

In [21]:
#Creates a histogram based on the STAAR/Percent Loss data, displays percent loss by district count
input_list = ['2019-2021','2021-2022','2019-2022']
class LossDashboard(param.Parameterized):
    _ = param.Selector(input_list,default=input_list[0])
    #@param.depends('_')
    def get_map(self):
        if self._ == '2019-2021':
            #data = loss_df[['% LOSS Math 2019 to 2021','% LOSS Reading 2019 to 2021']]
            fig = go.Figure(data=[go.Histogram(x=loss_df['% LOSS Math 2019 to 2021'],
                                      name='Math')])
            fig.add_trace(go.Histogram(x=loss_df['% LOSS Reading 2019 to 2021'],
                                      name='Reading'))
            fig.update_layout(barmode='stack')
        elif self._ == '2021-2022':
            #data = loss_df[['% LOSS Math 2021 to 2022','% LOSS Reading 2021 to 2022']]
            fig = go.Figure(data=[go.Histogram(x=loss_df['% LOSS Math 2021 to 2022'],
                                      name='Math')])
            fig.add_trace(go.Histogram(x=loss_df['% LOSS Reading 2021 to 2022'],
                                      name='Reading'))
            fig.update_layout(barmode='stack')
        elif self._ == '2019-2022':
            #data = loss_df[['% LOSS Math 2019 to 2022','% LOSS Reading 2019 to 2022']]
            fig = go.Figure(data=[go.Histogram(x=loss_df['% LOSS Math 2019 to 2022'],
                                      name='Math')])
            fig.add_trace(go.Histogram(x=loss_df['% LOSS Reading 2019 to 2022'],
                                      name='Reading'))
            fig.update_layout(barmode='stack')
            
        fig.update_xaxes(range=[-30,30],dtick=10)
        fig.update_yaxes(range=[0,600],dtick=100)
       # fig.update_traces(opacity=0.7)
        fig.update_layout(height=500, width =680,
                          xaxis_title_text='% Loss', 
                          yaxis_title_text='Number of Districts',
                          bargap=0.2, hovermode=False,
                          hoverlabel=dict(bgcolor='black',font_size=24,font_family="Times New Roman"),
        font=dict(family='times new roman',
        size=23,color='black'),
        title_x=0.55,
       # title_y=.95,
        margin={"r":10,"t":10,"l":0,"b":15})
        return fig
    def panel(self):
        return pn.Column(
            pn.Param(self,width=450,styles={'background': '#FFFFFF'},
                     widgets={'_':{'widget_type':pn.widgets.RadioButtonGroup}}),            
            self.get_map
        )
dashboard = LossDashboard(name = 'Normalized Difference in STAAR Scores, Avg. Over TX ISDs')
loss_component1 = dashboard.panel()
loss_component1.save('Histogram_Tab',resources=INLINE)

In [22]:
#Import dataframe
address = ("Data/District_Loss/heat_df.csv")
heat_df = pd.read_csv(address)
#Group the dataset by demographic group
heat_df= heat_df.set_index('group')

In [23]:
heat1921 = pd.read_excel("Data/District_Loss/heat1921.xlsx")
heat1921 = heat1921.set_index('group')

heat1922 = pd.read_excel("Data/District_Loss/heat1922.xlsx")
heat1922 = heat1922.set_index('group')

heat2122 = pd.read_excel("Data/District_Loss/heat2122.xlsx")
heat2122 = heat2122.set_index('group')

In [24]:
heat1921

,subject,3rd,4th,5th,6th,7th,8th
group,,,,,,,
All,Math,-0.048330,-0.036375,-0.003679,-0.016020,-0.073340,0.007618
Asian,Math,-0.054235,-0.044694,-0.008467,-0.030249,-0.081712,-0.004837
Black,Math,-0.062425,-0.046983,-0.013881,-0.020411,-0.074143,0.003834
Free Lunch,Math,-0.052935,-0.040087,-0.004892,-0.016930,-0.074014,0.006970
Hispanic,Math,-0.052544,-0.041044,-0.003416,-0.017272,-0.075841,0.005746
Poverty,Math,-0.051965,-0.039868,-0.004038,-0.017514,-0.073660,0.007424
Special Ed,Math,-0.040454,-0.032465,0.006543,-0.002194,-0.060570,0.018979
T1,Math,-0.049197,-0.037276,-0.003468,-0.016185,-0.073582,0.007722
All,Reading,-0.002829,0.009321,-0.014458,-0.001719,0.008341,0.003484


In [25]:
heat_df

,subject,3.0,4.0,5.0,6.0,7.0,8.0
group,,,,,,,
All,Math,0.054,0.041,0.011,0.023,0.071,0.006
Asian,Math,0.054,0.044,0.009,0.031,0.077,0.002
Black,Math,0.056,0.042,0.011,0.019,0.065,0.004
Free Lunch,Math,0.057,0.041,0.011,0.023,0.069,0.006
Hispanic,Math,0.056,0.043,0.012,0.024,0.071,0.007
Poverty,Math,0.056,0.042,0.012,0.023,0.070,0.007
Special Ed,Math,0.044,0.033,-0.001,0.006,0.051,-0.010
T1,Math,0.055,0.042,0.011,0.025,0.071,0.009
White,Math,0.045,0.032,0.008,0.018,0.070,0.001


In [26]:
heat1922['3rd'] = heat1922['3rd']*100
heat1922['4th'] = heat1922['4th']*100
heat1922['5th'] = heat1922['5th']*100
heat1922['6th'] = heat1922['6th']*100
heat1922['7th'] = heat1922['7th']*100
heat1922['8th'] = heat1922['8th']*100
heat1922=heat1922.round(2)

In [27]:
heat1921['3rd'] = heat1921['3rd']*100
heat1921['4th'] = heat1921['4th']*100
heat1921['5th'] = heat1921['5th']*100
heat1921['6th'] = heat1921['6th']*100
heat1921['7th'] = heat1921['7th']*100
heat1921['8th'] = heat1921['8th']*100
heat1921=heat1921.round(2)

In [28]:
heat2122['3rd'] = heat2122['3rd']*100
heat2122['4th'] = heat2122['4th']*100
heat2122['5th'] = heat2122['5th']*100
heat2122['6th'] = heat2122['6th']*100
heat2122['7th'] = heat2122['7th']*100
heat2122['8th'] = heat2122['8th']*100
heat2122=heat2122.round(2)

In [29]:
heat1922

,subject,3rd,4th,5th,6th,7th,8th
group,,,,,,,
All,Math,-1.19,4.13,2.40,-2.31,-9.15,0.64
Asian,Math,-0.96,3.67,1.57,-3.30,-9.62,0.57
Black,Math,-1.93,3.49,2.25,-2.51,-9.32,0.34
Free Lunch,Math,-1.42,3.88,2.57,-2.31,-9.22,0.53
Hispanic,Math,-1.42,3.96,2.65,-2.45,-9.28,0.57
Poverty,Math,-1.39,3.95,2.58,-2.28,-9.18,0.55
Special Ed,Math,-0.55,4.65,3.42,-0.84,-8.01,1.39
T1,Math,-1.15,4.15,2.51,-2.24,-9.08,0.80
All,Reading,-2.70,3.06,0.71,0.17,-0.81,0.60


In [30]:
heat_df['3.0'] = heat_df['3.0']*-1
heat_df['4.0'] = heat_df['4.0']*-1
heat_df['5.0'] = heat_df['5.0']*-1
heat_df['6.0'] = heat_df['6.0']*-1
heat_df['7.0'] = heat_df['7.0']*-1
heat_df['8.0'] = heat_df['8.0']*-1
heat_df['3.0'] = heat_df['3.0']*100
heat_df['4.0'] = heat_df['4.0']*100
heat_df['5.0'] = heat_df['5.0']*100
heat_df['6.0'] = heat_df['6.0']*100
heat_df['7.0'] = heat_df['7.0']*100
heat_df['8.0'] = heat_df['8.0']*100
heat_df=heat_df.round(2)
heat_df.rename(columns={'3.0':'3rd',
                      '4.0':'4th',
                      '5.0':'5th',
                      '6.0':'6th',
                      '7.0':'7th',
                      '8.0':'8th'},inplace=True)

In [31]:
heat_df

,subject,3rd,4th,5th,6th,7th,8th
group,,,,,,,
All,Math,-5.4,-4.1,-1.1,-2.3,-7.1,-0.6
Asian,Math,-5.4,-4.4,-0.9,-3.1,-7.7,-0.2
Black,Math,-5.6,-4.2,-1.1,-1.9,-6.5,-0.4
Free Lunch,Math,-5.7,-4.1,-1.1,-2.3,-6.9,-0.6
Hispanic,Math,-5.6,-4.3,-1.2,-2.4,-7.1,-0.7
Poverty,Math,-5.6,-4.2,-1.2,-2.3,-7.0,-0.7
Special Ed,Math,-4.4,-3.3,0.1,-0.6,-5.1,1.0
T1,Math,-5.5,-4.2,-1.1,-2.5,-7.1,-0.9
White,Math,-4.5,-3.2,-0.8,-1.8,-7.0,-0.1


In [32]:
#Creates a heatmap based off of the STAAR/Percent Loss Data by demographic group and grade
subject_inputs = ['Math','Reading']
year_list = ['2019-2021','2021-2022','2019-2022']
class LossDashboard(param.Parameterized):
    Subject = param.Selector(subject_inputs,default = subject_inputs[0])
    __ = param.Selector(year_list, default=year_list[0])
    #@param.depends('Subject')
    def lossplot(self):
        if self.__ == '2019-2021':
            data = heat1921[heat1921['subject'] == self.Subject]
        elif self.__ == '2019-2022':
            data = heat1922[heat1922['subject'] == self.Subject]
        elif self.__ == '2021-2022':
            data = heat2122[heat2122['subject'] == self.Subject]
#        data = heat_df[heat_df['subject'] == self.Subject]
        data = data.drop(['subject'],axis=1)
        fig = px.imshow(data, labels=dict(x='Grade',y='Demographic Group',color='Learning Loss %'),text_auto='.1f',
        height=450,width=800,
        color_continuous_scale='balance_r',
       # labels={'Learning Loss':'Learning Loss '}
        range_color=(-5,5),
        aspect='none')
        fig.update_layout(hoverlabel=dict(bgcolor='black',font_size=24,font_family="Times New Roman"),
        font=dict(family='times new roman',
        size=24,color='black'),
        title_x=.96,                
        margin={"r":10,"t":10,"l":0,"b":15})
        fig.update_coloraxes(cmid=0,colorbar_title_font_family='Times New Roman',colorbar_title_text='% Loss',
                             colorbar_title_font_size=24,colorbar_title_side='bottom',colorbar_thickness=10)
        return fig
    def panel(self):
        return pn.Column(
          #  pn.Param(self, width=350, name='Normalized Difference in STAAR Scores Between 2019 and 2021 By Demographic Group and Grade:',sizing_mode = 'fixed',background='#FFFFFF'),  
            pn.Param(self,width=550,styles={'background': '#FFFFFF'},
            widgets={'__':{'widget_type':pn.widgets.RadioButtonGroup}}),
            self.lossplot
        )
dashboard = LossDashboard(name = 'Normalized Difference in STAAR Scores By Demographic Group and Grade')
loss_component2 = dashboard.panel()
loss_component2

BokehModel(combine_events=True, render_bundle={'docs_json': {'69018077-988d-45d4-bc88-84737b0da347': {'version…

# Texas District Level Maps

In [33]:
tx_map = gpd.read_file("Data/ISDmaps/Districts2020to2021.geojson")
label_data = pd.read_csv("Data/District_Loss/DATA_Texas_District_v1.csv")

In [34]:
label_data["NCES_DISTR"] = label_data["NCES_DISTR"].astype(int)
label_data = label_data.sort_values("NCES_DISTR")
tx_map = tx_map.sort_values("NCES_DISTR")
tx_map['NCES_DISTR'] = tx_map['NCES_DISTR'].astype(int)

In [35]:
merged_df = tx_map.merge(label_data, how = 'inner', on=['NCES_DISTR','DISTRICT_N'])

In [36]:
merged_df['Full-Time Equivalent(FTE) Teachers Difference (2019-2021)'] = (merged_df['FTE Teachers 2020-2021']-merged_df['FTE Teachers 2018-2019'])/merged_df['FTE Teachers 2018-2019']
merged_df['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2021)'] = (merged_df['Total Staff 2020-2021']-merged_df['Total Staff 2018-2019'])/merged_df['Total Staff 2018-2019']
merged_df['Total Number of Students Difference (2019-2021)'] = (merged_df['Total Students 2020-2021']-merged_df['Total Students 2018-2019'])/merged_df['Total Students 2018-2019']

merged_df['Full-Time Equivalent(FTE) Teachers Difference (2021-2022)'] = (merged_df['FTE Teachers 2021-2022']-merged_df['FTE Teachers 2020-2021'])/merged_df['FTE Teachers 2020-2021']
merged_df['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2022)'] = (merged_df['Total Staff 2021-2022']-merged_df['Total Staff 2020-2021'])/merged_df['Total Staff 2020-2021']
merged_df['Total Number of Students Difference (2021-2022)'] = (merged_df['Total Students 2021-2022']-merged_df['Total Students 2020-2021'])/merged_df['Total Students 2020-2021']

merged_df['Full-Time Equivalent(FTE) Teachers Difference (2019-2022)'] = (merged_df['FTE Teachers 2021-2022']-merged_df['FTE Teachers 2018-2019'])/merged_df['FTE Teachers 2018-2019']
merged_df['Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2022)'] = (merged_df['Total Staff 2021-2022']-merged_df['Total Staff 2018-2019'])/merged_df['Total Staff 2018-2019']
merged_df['Total Number of Students Difference (2019-2022)'] = (merged_df['Total Students 2021-2022']-merged_df['Total Students 2018-2019'])/merged_df['Total Students 2018-2019']

In [37]:
merged_df = merged_df.round(2)

In [38]:
merged_df.rename(columns={'% LOSS Math 2019 to 2021':'Average Score Difference(%) in Math (2019-2021)',
                          '% LOSS Reading 2019 to 2021':'Average Score Difference(%) in Reading (2019-2021)',
                          '% LOSS All 2019 to 2021':'Average Score Difference(%) in All (2019-2021)',
                          'Label_Math (19-21)':'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2019-2021)',
                          'Label_Reading (19-21)':'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2019-2021)',
                          'Label_All (19-21)':'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2019-2021)'},inplace=True)

merged_df.rename(columns={'% LOSS Math 2021 to 2022':'Average Score Difference(%) in Math (2021-2022)',
                          '% LOSS Reading 2021 to 2022':'Average Score Difference(%) in Reading (2021-2022)',
                          '% LOSS All 2021 to 2022':'Average Score Difference(%) in All (2021-2022)',
                          'Label_Math (21-22)':'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2021-2022)',
                          'Label_Reading (21-22)':'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2021-2022)',
                          'Label_All (21-22)':'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2021-2022)'},inplace=True)

merged_df.rename(columns={'% LOSS Math 2019 to 2022':'Average Score Difference(%) in Math (2019-2022)',
                          '% LOSS Reading 2019 to 2022':'Average Score Difference(%) in Reading (2019-2022)',
                          '% LOSS All 2019 to 2022':'Average Score Difference(%) in All (2019-2022)',
                          'Label_Math (19-22)':'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2019-2022)',
                          'Label_Reading (19-22)':'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2019-2022)',
                          'Label_All (19-22)':'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2019-2022)'},inplace=True)

In [39]:
county_dict = pd.DataFrame()
county_dict['County #'] = comp_df['County #']
county_dict['County Name'] = comp_df['County Name']
cdict = pd.Series(county_dict['County #'].values,index=county_dict['County Name']).to_dict()
ndict = pd.Series(county_dict['County Name'].values,index=county_dict['County #']).to_dict()
label_data['County Number'] = label_data['County #']
label_data.replace({'County #': ndict}, inplace=True)
label_data.rename(columns = {'County #': 'County Name','County Number':'County #'}, inplace=True)

In [40]:
cdict

{'Anderson': 48001,
 'Andrews': 48003,
 'Angelina': 48005,
 'Aransas': 48007,
 'Archer': 48009,
 'Armstrong': 48011,
 'Atascosa': 48013,
 'Austin': 48015,
 'Bailey': 48017,
 'Bandera': 48019,
 'Bastrop': 48021,
 'Baylor': 48023,
 'Bee': 48025,
 'Bell': 48027,
 'Bexar': 48029,
 'Blanco': 48031,
 'Borden': 48033,
 'Bosque': 48035,
 'Bowie': 48037,
 'Brazoria': 48039,
 'Brazos': 48041,
 'Brewster': 48043,
 'Briscoe': 48045,
 'Brooks': 48047,
 'Brown': 48049,
 'Burleson': 48051,
 'Burnet': 48053,
 'Caldwell': 48055,
 'Calhoun': 48057,
 'Callahan': 48059,
 'Cameron': 48061,
 'Camp': 48063,
 'Carson': 48065,
 'Cass': 48067,
 'Castro': 48069,
 'Chambers': 48071,
 'Cherokee': 48073,
 'Childress': 48075,
 'Clay': 48077,
 'Cochran': 48079,
 'Coke': 48081,
 'Coleman': 48083,
 'Collin': 48085,
 'Collingsworth': 48087,
 'Colorado': 48089,
 'Comal': 48091,
 'Comanche': 48093,
 'Concho': 48095,
 'Cooke': 48097,
 'Coryell': 48099,
 'Cottle': 48101,
 'Crane': 48103,
 'Crockett': 48105,
 'Crosby': 48107

In [41]:
county_dict.to_csv('countyfipsmatcher.csv')

In [42]:
merged_df.rename(columns={'Full-Time Equivalent(FTE) Teachers Difference (2019-2021)':'Number of Teachers (2019-2021)',
                          'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2021)':'Number of Overall Staff (2019-2021)',
                          'Total Number of Students Difference (2019-2021)': 'Number of Students (2019-2021)',
                          'Average Score Difference(%) in Math (2019-2021)': 'STAAR Score in Math (2019-2021)',
                          'Average Score Difference(%) in Reading (2019-2021)': 'STAAR Score in Reading (2019-2021)',
                          'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2019-2021)': 'Learning Loss in Math (2019-2021)',
                          'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2019-2021)': 'Learning Loss in Reading (2019-2021)',
                          'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2019-2021)': 'Learning Loss (2019-2021)'},inplace=True)

merged_df.rename(columns={'Full-Time Equivalent(FTE) Teachers Difference (2021-2022)':'Number of Teachers (2021-2022)',
                          'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2021-2022)':'Number of Overall Staff (2021-2022)',
                          'Total Number of Students Difference (2021-2022)': 'Number of Students (2021-2022)',
                          'Average Score Difference(%) in Math (2021-2022)': 'STAAR Score in Math (2021-2022)',
                          'Average Score Difference(%) in Reading (2021-2022)': 'STAAR Score in Reading (2021-2022)',
                          'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2021-2022)': 'Learning Loss in Math (2021-2022)',
                          'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2021-2022)': 'Learning Loss in Reading (2021-2022)',
                          'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2021-2022)': 'Learning Loss (2021-2022)'},inplace=True)

merged_df.rename(columns={'Full-Time Equivalent(FTE) Teachers Difference (2019-2022)':'Number of Teachers (2019-2022)',
                          'Sum of Full-Time Equivalent(FTE) Teachers and Staffs Difference (2019-2022)':'Number of Overall Staff (2019-2022)',
                          'Total Number of Students Difference (2019-2022)': 'Number of Students (2019-2022)',
                          'Average Score Difference(%) in Math (2019-2022)': 'STAAR Score in Math (2019-2022)',
                          'Average Score Difference(%) in Reading (2019-2022)': 'STAAR Score in Reading (2019-2022)',
                          'Learning Loss in Math: (-1) Loss (0) Expected (1) Gain (2019-2022)': 'Learning Loss in Math (2019-2022)',
                          'Learning Loss in Reading: (-1) Loss (0) Expected (1) Gain (2019-2022)': 'Learning Loss in Reading (2019-2022)',
                          'Learning Loss in Math and Reading: (-1) Loss (0) Expected (1) Gain (2019-2022)': 'Learning Loss (2019-2022)'},inplace=True)

In [43]:
merged_df['Number of Teachers (2019-2021)'] = merged_df['Number of Teachers (2019-2021)']*100
merged_df['Number of Overall Staff (2019-2021)'] = merged_df['Number of Overall Staff (2019-2021)']*100
merged_df['Number of Students (2019-2021)'] = merged_df['Number of Students (2019-2021)']*100
merged_df['STAAR Score in Math (2019-2021)'] = merged_df['STAAR Score in Math (2019-2021)']*100
merged_df['STAAR Score in Reading (2019-2021)'] = merged_df['STAAR Score in Reading (2019-2021)']*100
merged_df['Math% (2019-2021)'] = merged_df['Learning Loss in Math (2019-2021)']
merged_df['Reading% (2019-2021)'] = merged_df['Learning Loss in Reading (2019-2021)']
merged_df['Loss% (2019-2021)'] = merged_df['Learning Loss (2019-2021)']

merged_df['Number of Teachers (2021-2022)'] = merged_df['Number of Teachers (2021-2022)']*100
merged_df['Number of Overall Staff (2021-2022)'] = merged_df['Number of Overall Staff (2021-2022)']*100
merged_df['Number of Students (2021-2022)'] = merged_df['Number of Students (2021-2022)']*100
merged_df['STAAR Score in Math (2021-2022)'] = merged_df['STAAR Score in Math (2021-2022)']*100
merged_df['STAAR Score in Reading (2021-2022)'] = merged_df['STAAR Score in Reading (2021-2022)']*100
merged_df['Math% (2021-2022)'] = merged_df['Learning Loss in Math (2021-2022)']
merged_df['Reading% (2021-2022)'] = merged_df['Learning Loss in Reading (2021-2022)']
merged_df['Loss% (2021-2022)'] = merged_df['Learning Loss (2021-2022)']

merged_df['Number of Teachers (2019-2022)'] = merged_df['Number of Teachers (2019-2022)']*100
merged_df['Number of Overall Staff (2019-2022)'] = merged_df['Number of Overall Staff (2019-2022)']*100
merged_df['Number of Students (2019-2022)'] = merged_df['Number of Students (2019-2022)']*100
merged_df['STAAR Score in Math (2019-2022)'] = merged_df['STAAR Score in Math (2019-2022)']*100
merged_df['STAAR Score in Reading (2019-2022)'] = merged_df['STAAR Score in Reading (2019-2022)']*100
merged_df['Math% (2019-2022)'] = merged_df['Learning Loss in Math (2019-2022)']
merged_df['Reading% (2019-2022)'] = merged_df['Learning Loss in Reading (2019-2022)']
merged_df['Loss% (2019-2022)'] = merged_df['Learning Loss (2019-2022)']

In [44]:
def lossfuncmath(row):
    if row['Learning Loss in Math (2019-2021)']>0:
        val = 'Gain'
    elif row['Learning Loss in Math (2019-2021)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Math (2019-2021)'] = merged_df.apply(lossfuncmath, axis=1)

def lossfuncreading(row):
    if row['Learning Loss in Reading (2019-2021)']>0:
        val = 'Gain'
    elif row['Learning Loss in Reading (2019-2021)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Reading (2019-2021)'] = merged_df.apply(lossfuncreading, axis=1)

def lossfunc(row):
    if row['Learning Loss (2019-2021)']>0:
        val = 'Gain'
    elif row['Learning Loss (2019-2021)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Overall (2019-2021)'] = merged_df.apply(lossfunc, axis=1)


###############################################################


def lossfuncmath(row):
    if row['Learning Loss in Math (2021-2022)']>0:
        val = 'Gain'
    elif row['Learning Loss in Math (2021-2022)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Math (2021-2022)'] = merged_df.apply(lossfuncmath, axis=1)

def lossfuncreading(row):
    if row['Learning Loss in Reading (2021-2022)']>0:
        val = 'Gain'
    elif row['Learning Loss in Reading (2021-2022)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Reading (2021-2022)'] = merged_df.apply(lossfuncreading, axis=1)

def lossfunc(row):
    if row['Learning Loss (2021-2022)']>0:
        val = 'Gain'
    elif row['Learning Loss (2021-2022)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Overall (2021-2022)'] = merged_df.apply(lossfunc, axis=1)


###############################################################


def lossfuncmath(row):
    if row['Learning Loss in Math (2019-2022)']>0:
        val = 'Gain'
    elif row['Learning Loss in Math (2019-2022)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Math (2019-2022)'] = merged_df.apply(lossfuncmath, axis=1)

def lossfuncreading(row):
    if row['Learning Loss in Reading (2019-2022)']>0:
        val = 'Gain'
    elif row['Learning Loss in Reading (2019-2022)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Reading (2019-2022)'] = merged_df.apply(lossfuncreading, axis=1)

def lossfunc(row):
    if row['Learning Loss (2019-2022)']>0:
        val = 'Gain'
    elif row['Learning Loss (2019-2022)']<0:
        val = 'Loss'
    else:
        val = 'No Unexpected Change'
    return val

merged_df['Label Overall (2019-2022)'] = merged_df.apply(lossfunc, axis=1)

In [45]:
merged_df.to_csv('Data/DistrictLevelstats.csv')

In [46]:
#merged_df['Number of Overall Staff'] = merged_df['Number of Overall Staff'].astype(int)
#merged_df['Number of Students'] = merged_df['Number of Students'].astype(int)
#merged_df['Number of Teachers'] = merged_df['Number of Teachers'].astype(int)
#merged_df['STAAR Score in Math'] = merged_df['STAAR Score in Math'].astype(int)
#merged_df['STAAR Score in Reading'] = merged_df['STAAR Score in Reading'].astype(int)

merged_df['Number of Overall Staff (2019-2021)'] = merged_df['Number of Overall Staff (2019-2021)'].astype(int)
merged_df['Number of Students (2019-2021)'] = merged_df['Number of Students (2019-2021)'].astype(int)
merged_df['Number of Teachers (2019-2021)'] = merged_df['Number of Teachers (2019-2021)'].astype(int)
merged_df['STAAR Score in Math (2019-2021)'] = merged_df['STAAR Score in Math (2019-2021)'].astype(int)
merged_df['STAAR Score in Reading (2019-2021)'] = merged_df['STAAR Score in Reading (2019-2021)'].astype(int)

merged_df['Number of Overall Staff (2021-2022)'] = merged_df['Number of Overall Staff (2021-2022)'].astype(int)
merged_df['Number of Students (2021-2022)'] = merged_df['Number of Students (2021-2022)'].astype(int)
merged_df['Number of Teachers (2021-2022)'] = merged_df['Number of Teachers (2021-2022)'].astype(int)
merged_df['STAAR Score in Math (2021-2022)'] = merged_df['STAAR Score in Math (2021-2022)'].astype(int)
merged_df['STAAR Score in Reading (2021-2022)'] = merged_df['STAAR Score in Reading (2021-2022)'].astype(int)

merged_df['Number of Overall Staff (2019-2022)'] = merged_df['Number of Overall Staff (2019-2022)'].astype(int)
merged_df['Number of Students (2019-2022)'] = merged_df['Number of Students (2019-2022)'].astype(int)
merged_df['Number of Teachers (2019-2022)'] = merged_df['Number of Teachers (2019-2022)'].astype(int)
merged_df['STAAR Score in Math (2019-2022)'] = merged_df['STAAR Score in Math (2019-2022)'].astype(int)
merged_df['STAAR Score in Reading (2019-2022)'] = merged_df['STAAR Score in Reading (2019-2022)'].astype(int)

In [47]:
df_1921 = merged_df[['NAME','County #','DISTRICT_N','Number of Teachers (2019-2021)',
                    'Number of Overall Staff (2019-2021)',
                    'Number of Students (2019-2021)',
                    'STAAR Score in Math (2019-2021)',
                    'STAAR Score in Reading (2019-2021)',
                    'Math% (2019-2021)',
                    'Reading% (2019-2021)',
                    'Loss% (2019-2021)']]

df_2122 = merged_df[['NAME','County #','DISTRICT_N','Number of Teachers (2021-2022)',
                    'Number of Overall Staff (2021-2022)',
                    'Number of Students (2021-2022)',
                    'STAAR Score in Math (2021-2022)',
                    'STAAR Score in Reading (2021-2022)',
                    'Math% (2021-2022)',
                    'Reading% (2021-2022)',
                    'Loss% (2021-2022)']]

df_1922 = merged_df[['NAME','County #','DISTRICT_N','Number of Teachers (2019-2022)',
                    'Number of Overall Staff (2019-2022)',
                    'Number of Students (2019-2022)',
                    'STAAR Score in Math (2019-2022)',
                    'STAAR Score in Reading (2019-2022)',
                    'Math% (2019-2022)',
                    'Reading% (2019-2022)',
                    'Loss% (2019-2022)']]

In [48]:
df_1921.rename(columns = {'Number of Teachers (2019-2021)':'Number of Teachers',
                           'Number of Overall Staff (2019-2021)':'Number of Staff',
                           'Number of Students (2019-2021)':'Number of Students',
                           'STAAR Score in Math (2019-2021)':'STAAR Score in Math',
                           'STAAR Score in Reading (2019-2021)':'STAAR Score in Reading',
                           'Math% (2019-2021)':'Math%',
                           'Reading% (2019-2021)':'Reading%',
                           'Loss% (2019-2021)':'Loss%'}, inplace=True)

df_2122.rename(columns = {'Number of Teachers (2021-2022)':'Number of Teachers',
                           'Number of Overall Staff (2021-2022)':'Number of Staff',
                           'Number of Students (2021-2022)':'Number of Students',
                           'STAAR Score in Math (2021-2022)':'STAAR Score in Math',
                           'STAAR Score in Reading (2021-2022)':'STAAR Score in Reading',
                           'Math% (2021-2022)':'Math%',
                           'Reading% (2021-2022)':'Reading%',
                           'Loss% (2021-2022)':'Loss%'}, inplace=True)

df_1922.rename(columns = {'Number of Teachers (2019-2022)':'Number of Teachers',
                           'Number of Overall Staff (2019-2022)':'Number of Staff',
                           'Number of Students (2019-2022)':'Number of Students',
                           'STAAR Score in Math (2019-2022)':'STAAR Score in Math',
                           'STAAR Score in Reading (2019-2022)':'STAAR Score in Reading',
                           'Math% (2019-2022)':'Math%',
                           'Reading% (2019-2022)':'Reading%',
                           'Loss% (2019-2022)':'Loss%'}, inplace=True)

C:\Users\Leaf\AppData\Local\Temp\ipykernel_23488\4287373170.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Leaf\AppData\Local\Temp\ipykernel_23488\4287373170.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Leaf\AppData\Local\Temp\ipykernel_23488\4287373170.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [49]:
merged_df['Total Funding'] = merged_df['CARES ESSER I 20'] + merged_df['CRRSA ESSER II 21'] + merged_df['ARP ESSER III 21'] + merged_df['ESSER-SUPP 22'] + merged_df['ESSER-SUPP 23']
merged_df['Funding Per Student'] = merged_df['Total Funding']/merged_df['Total Students 2021-2022']

In [50]:
merged_df['CARES ESSER I 20'] = merged_df['CARES ESSER I 20'].round(2)
merged_df['CRRSA ESSER II 21'] = merged_df['CRRSA ESSER II 21'].round(2)
merged_df['ARP ESSER III 21'] = merged_df['ARP ESSER III 21'].round(2)
merged_df['ESSER-SUPP 22'] = merged_df['ESSER-SUPP 22'].round(2)
merged_df['ESSER-SUPP 23'] = merged_df['ESSER-SUPP 23'].round(2)
merged_df['Total Funding'] = merged_df['Total Funding'].round(2)
merged_df['Funding Per Student'] = merged_df['Funding Per Student'].round(2)

In [51]:
merged_df['CARES ESSER DOLLAR'] = merged_df['CARES ESSER I 20'].apply(lambda x: "${:0,.0f}".format(x))
merged_df['CRRSA ESSER DOLLAR'] = merged_df['CRRSA ESSER II 21'].apply(lambda x: "${:0,.0f}".format(x))
merged_df['ARP ESSER DOLLAR'] = merged_df['ARP ESSER III 21'].apply(lambda x: "${:0,.0f}".format(x))
merged_df['ESSER 22 DOLLAR'] = merged_df['ESSER-SUPP 22'].apply(lambda x: "${:0,.0f}".format(x))
merged_df['ESSER 23 DOLLAR'] = merged_df['ESSER-SUPP 23'].apply(lambda x: "${:0,.0f}".format(x))
merged_df['PER STUDENT DOLLAR'] = merged_df['Funding Per Student'].apply(lambda x: "${:0,.0f}".format(x))
merged_df['TOTAL DOLLAR'] = merged_df['Total Funding'].apply(lambda x: "${:0,.0f}".format(x))

In [52]:
cdict

{'Anderson': 48001,
 'Andrews': 48003,
 'Angelina': 48005,
 'Aransas': 48007,
 'Archer': 48009,
 'Armstrong': 48011,
 'Atascosa': 48013,
 'Austin': 48015,
 'Bailey': 48017,
 'Bandera': 48019,
 'Bastrop': 48021,
 'Baylor': 48023,
 'Bee': 48025,
 'Bell': 48027,
 'Bexar': 48029,
 'Blanco': 48031,
 'Borden': 48033,
 'Bosque': 48035,
 'Bowie': 48037,
 'Brazoria': 48039,
 'Brazos': 48041,
 'Brewster': 48043,
 'Briscoe': 48045,
 'Brooks': 48047,
 'Brown': 48049,
 'Burleson': 48051,
 'Burnet': 48053,
 'Caldwell': 48055,
 'Calhoun': 48057,
 'Callahan': 48059,
 'Cameron': 48061,
 'Camp': 48063,
 'Carson': 48065,
 'Cass': 48067,
 'Castro': 48069,
 'Chambers': 48071,
 'Cherokee': 48073,
 'Childress': 48075,
 'Clay': 48077,
 'Cochran': 48079,
 'Coke': 48081,
 'Coleman': 48083,
 'Collin': 48085,
 'Collingsworth': 48087,
 'Colorado': 48089,
 'Comal': 48091,
 'Comanche': 48093,
 'Concho': 48095,
 'Cooke': 48097,
 'Coryell': 48099,
 'Cottle': 48101,
 'Crane': 48103,
 'Crockett': 48105,
 'Crosby': 48107

In [53]:
#Creates a county map of Texas based off of the National Center on Education Statistics DataFrame
input_list = ['STAAR Score in Math',
              'STAAR Score in Reading',
              'Number of Students',
              'Number of Teachers',
              'Number of Staff'
             ]
funding_list = ['CARES ESSER I 20',
              'CRRSA ESSER II 21',
              'ARP ESSER III 21',
              'ESSER-SUPP 22',
              'ESSER-SUPP 23',
              'Funding Per Student'
              'Total Funding']
year_list = ['2019-2021','2021-2022','2019-2022','Funding']
district_list = label_data['County Name'].values
district_list = sorted(district_list)
class MapDashboard(param.Parameterized):
    

    __   = param.Selector(default='Hays',objects=district_list,check_on_set=False)
    ___  = param.Selector(default=year_list[0],objects=year_list)
    _    = param.ObjectSelector(default=input_list[0],objects=input_list)
    lists = {'2019-2021':['STAAR Score in Math','STAAR Score in Reading','Number of Students','Number of Teachers','Number of Staff'],
             '2021-2022':['STAAR Score in Math','STAAR Score in Reading','Number of Students','Number of Teachers','Number of Staff'],
             '2019-2022':['STAAR Score in Math','STAAR Score in Reading','Number of Students','Number of Teachers','Number of Staff'],
             'Funding':['CARES ESSER I 20','CRRSA ESSER II 21','ARP ESSER III 21','ESSER-SUPP 22','ESSER-SUPP 23','Funding Per Student','Total Funding']
            }
    @param.depends('___',watch=True)
    def update_list(self):
        up_list = self.lists[self.___]
        self.param['_'].objects = up_list
        self._ = up_list[0]
        
    def get_map(self):
        county_num = cdict[self.__]
        if self.___ == '2019-2021':
            yr_df = df_1921.loc[df_1921['County #'] == county_num]
            data = yr_df[['NAME','County #','DISTRICT_N',
                     #  'Label Math ','Label Reading ','Label Overall ', 
                       'Math%','Reading%','Loss%', self._]]
        elif self.___ == '2021-2022':
            yr_df = df_2122.loc[df_2122['County #'] == county_num]
            data = yr_df[['NAME','County #','DISTRICT_N',
                     #  'Label Math ','Label Reading ','Label Overall ', 
                       'Math%','Reading%','Loss%', self._]]
        elif self.___ == '2019-2022':
            yr_df = df_1922.loc[df_1922['County #'] == county_num]
            data = yr_df[['NAME','County #','DISTRICT_N',
                     #  'Label Math ','Label Reading ','Label Overall ', 
                       'Math%','Reading%','Loss%', self._]]
        elif self.___ == 'Funding':
            yr_df = merged_df.loc[merged_df['County #'] == county_num]
            data = yr_df[['NAME','County #','DISTRICT_N',
                          'CARES ESSER DOLLAR','CRRSA ESSER DOLLAR','ARP ESSER DOLLAR',
                          'ESSER 22 DOLLAR','ESSER 23 DOLLAR','PER STUDENT DOLLAR','TOTAL DOLLAR',self._]]
        
        dis_df = merged_df.loc[merged_df['County #'] == county_num]
        
        if self.___ == 'Funding':
            if self._ == 'CARES ESSER I 20':
                minimum = 0
                maximum = 2000000
            elif self._ == 'CRRSA ESSER II 21':
                minimum = 0
                maximum = 9000000
            elif self._ == 'ARP ESSER III 21':
                minimum = 0
                maximum = 20000000
            elif self._ == 'ESSER-SUPP 22':
                minimum = 0
                maximum = 200000
            elif self._ == 'ESSER-SUPP 23':
                minimum = 0
                maximum = 200000
            elif self._ == 'Funding Per Student':
                minimum = 0
                maximum = 6000
            else:
                minimum = 0
                maximum = 32000000
            fig = px.choropleth(data, geojson=dis_df, locations='DISTRICT_N', color= self._,
            color_continuous_scale='greens',
            range_color=(minimum,maximum),
            featureidkey='properties.DISTRICT_N',
            title =self.__ + ' County',
            custom_data = [data['NAME'],self._,
                           data['CARES ESSER DOLLAR'],data['CRRSA ESSER DOLLAR'],data['ARP ESSER DOLLAR'],
                          data['ESSER 22 DOLLAR'],data['ESSER 23 DOLLAR'],data['PER STUDENT DOLLAR'],data['TOTAL DOLLAR']],
            height=550, width=480)
            if self._ == 'CARES ESSER I 20':
                fig.update_layout(coloraxis_colorbar=dict(tickvals=[100000,550000,1000000,1450000,1900000],ticktext=['$0','$0.5M','$1M','$1.5M','$2M+']))
            elif self._ == 'CRRSA ESSER II 21':
                fig.update_layout(coloraxis_colorbar=dict(tickvals=[170000,2300000,4500000,6700000,8800000],ticktext=['$0','$2.25M','$4.5M','$6.75M','$9M+']))
            elif self._ == 'ARP ESSER III 21':
                fig.update_layout(coloraxis_colorbar=dict(tickvals=[1000000,5050000,10000000,14950000,19200000],ticktext=['$0','$5M','$10M','$15M','$20M+']))
            elif self._ == 'ESSER-SUPP 22':
                fig.update_layout(coloraxis_colorbar=dict(tickvals=[7000,50500,100000,149500,192000],ticktext=['$0','$50K','$100K','$150K','$200K+']))
            elif self._ == 'ESSER-SUPP 23':
                fig.update_layout(coloraxis_colorbar=dict(tickvals=[7000,50500,100000,149500,192000],ticktext=['$0','$50K','$100K','$150K','$200K+']))
            elif self._ == 'Funding Per Student':
                fig.update_layout(coloraxis_colorbar=dict(tickvals=[200,1550,3000,4450,5750],ticktext=['$0','$1500','$3000','$4500','$6000+']))
            elif self._ == 'Total Funding':
                fig.update_layout(coloraxis_colorbar=dict(tickvals=[1000000,8500000,16000000,23500000,31000000],ticktext=['$0','$8M','$16M','$24M','$32M+']))

        else:
            minimum = -10
            maximum = 10
            fig = px.choropleth(data, geojson=dis_df, locations='DISTRICT_N', color= self._,
            color_continuous_scale='balance_r',
            range_color=(minimum,maximum),
            featureidkey='properties.DISTRICT_N',
            title =self.__ + ' County',
            custom_data = [data['NAME'],self._,data['Math%'],data['Reading%'],data['Loss%']],
            height=550, width=480
                               )
            fig.update_layout(coloraxis_colorbar=dict(tickvals=[-8,-4,0,4,8],ticktext=['-10','-5','0','5','10']))
            
        fig.update_coloraxes(cmid=0,colorbar_orientation='h',colorbar_y=-0.1,colorbar_title_font_family='Times New Roman',
                             colorbar_title_font_size=22,colorbar_title_side='bottom',colorbar_thickness=4)

   #     fig.update_coloraxes(cmid=0,colorbar_orientation='h',colorbar_y=-0.1,colorbar_title_font_family='Times New Roman',
   #                          colorbar_title_font_size=24,colorbar_title_side='bottom',colorbar_thickness=4)
        if self.___ == 'Funding':
            if self._ == 'Total Funding':
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '%{customdata[8]} in ' + self._
            elif self._ == 'CARES ESSER I 20':
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '%{customdata[2]} from ' + self._
            elif self._ == 'CRRSA ESSER II 21':
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '%{customdata[3]} from ' + self._
            elif self._ == 'ARP ESSER III 21':
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '%{customdata[4]} from ' + self._
            elif self._ == 'ESSER-SUPP 22':
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '%{customdata[5]} from ' + self._
            elif self._ == 'ESSER-SUPP 23':
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '%{customdata[6]} from ' + self._
            elif self._ == 'Funding Per Student':
                hovertemp = ''
                hovertemp += '%{customdata[0]}<br>'
                hovertemp += '%{customdata[7]} in ' + self._

        else:
            hovertemp = ''
            hovertemp += '%{customdata[0]}<br>'
            hovertemp += '%{customdata[1]}% change in ' + self._
        
            
        fig.update_traces(hovertemplate=hovertemp)
        fig.update_geos(fitbounds='locations',visible=False)
        fig.update_layout(hoverlabel=dict(bgcolor='black',font_size=24,font_family="Times New Roman"),
        font=dict(family='times new roman',size=23,color='black'),
        title_x=.5,
        title_y=.97,                  
        margin={"r":10,"t":0,"l":0,"b":0})
        return fig
        
    def get_description(self):
        descriptions = {
            "2019-2021": {
                "STAAR Score in Math": "The percent difference in <b>Math STAAR scores</b> between <b>2019 and 2021</b> for a district.",
                "STAAR Score in Reading": "The percent difference in <b>Reading STAAR scores</b> between <b>2019 and 2021</b> for a district.",
                "Number of Teachers": "The percent difference in <b>Full-Time Teachers</b> between <b>2019 and 2021</b> for a district.",
                "Number of Staff": "The percent difference in <b>staff</b> between <b>2019 and 2021</b> for a district.",
                "Number of Students": "The percent difference in <b>students</b> between <b>2019 and 2021</b> for a district."
            },
            "2021-2022": {
                "STAAR Score in Math": "The percent difference in <b>Math STAAR scores</b> between <b>2021 and 2022</b> for a district.",
                "STAAR Score in Reading": "The percent difference in <b>Reading STAAR scores</b> between <b>2021 and 2022</b> for a district.",
                "Number of Teachers": "The percent difference in <b>Full-Time Teachers</b> between <b>2021 and 2022</b> for a district.",
                "Number of Staff": "The percent difference in <b>staff</b> between <b>2021 and 2022</b> for a district.",
                "Number of Students": "The percent difference in <b>students</b> between <b>2021 and 2022</b> for a district."
            },
            "2019-2022": {
                "STAAR Score in Math": "The percent difference in <b>Math STAAR scores</b> between <b>2019 and 2022</b> for a district.",
                "STAAR Score in Reading": "The percent difference in <b>Reading STAAR scores</b> between <b>2019 and 2022</b> for a district.",
                "Number of Teachers": "The percent difference in <b>Full-Time Teachers</b> between <b>2019 and 2022</b> for a district.",
                "Number of Staff": "The percent difference in <b>staff</b> between <b>2019 and 2022</b> for a district.",
                "Number of Students": "The percent difference in <b>students</b> between <b>2019 and 2022</b> for a district."
            },
            "Funding": {
                "CARES ESSER I 20": "The dollar amount of funding granted to a district through the <b>CARES ESSER I 20 Program</b>.",
                "CRRSA ESSER II 21": "The dollar amount of funding granted to a district through the <b>CRRSA ESSER II 21 Program</b>.",
                "ARP ESSER III 21": "The dollar amount of funding granted to a district through the <b>ARP ESSER III 21 Program</b>.",
                "ESSER-SUPP 22": "The dollar amount of funding granted to a district through the <b>ESSER-SUPP 22 Program</b>.",
                "ESSER-SUPP 23": "The dollar amount of funding granted to a district through the <b>ESSER-SUPP 23 Program</b>.",
                "Funding Per Student": "The total dollar amount of funding per student granted to a district.",
                "Total Funding": "The total dollar amount of funding granted to a district."
            }
        }
        if self.___ in descriptions:
            description_text = descriptions[self.___].get(self._)
            if description_text:
                return pn.pane.HTML(description_text, 
                                    styles={'color':'black','font-size':'20px','width':'125%',
                                            'text-align':'center', 'max-width':'500px','word-wrap':'break-word',
                                            'margin-left': '147px', 'font-family':'Times New Roman'})
        return None
    
    
    def panel(self):
        return pn.Row(
            pn.Column(
            pn.pane.HTML("""<h1>Select A County & Variable To View At The District Level </h1>""",
                              styles={'color':'black','font-size':'14px','width':'100%',
                                     'text-align':'center',
                                     'font-family':'Times New Roman'}),
            
            # We're using a basic margin-left to center the widget in lieu the grid system due to ease of use, since this is a static dashboard.
            pn.Param(self,width=550,styles={'background': '#FFFFFF', 'margin-left': '213px'},
                     widgets={'__':{'widget_type':pn.widgets.AutocompleteInput,'placeholder':'Enter a county name...'},
                             '___':{'widget_type':pn.widgets.RadioButtonGroup},}
                    ),
            pn.Spacer(height=50),
            pn.pane.HTML("""<h1>What does this variable show us?</h1>""",
                            styles={'color':'black','font-size':'14px','width':'100%',
                                  'text-align':'center',
                                  'font-family':'Times New Roman'}),
            pn.Spacer(height=5),
            self.get_description,
            ),
            pn.Spacer(width=150),
            self.get_map
        )
dashboard = MapDashboard(name='')
map_component2 = dashboard.panel()
map_component2.save('DistrictLevelTab',resources=INLINE)

# County Level Dashboard

In [54]:
#Panel HTML elements can take a 'styles' parameter directly. 
#We are alternatively assigning a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                               css_classes=['title-html-pane'],
                               styles={'color':'white','font-size':'14px','width':'100%',
                                       'background-color':'#6A5638','text-align':'center',
                                       'outline-style':'solid','outline-color':'#30231D',
                                       'outline-width':'thick','font-family':'Times New Roman',
                                       'height':'140px','padding':'10px','border-radius':'10px'})

datalab_img = pn.pane.PNG("Data/Images/datalabimage.png", height=140, width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png", height=140, width=350)

image_row = pn.Row(
    datalab_img,
    title_html_pane,
    txst_img,
    css_classes=['image-box']
)

content_bar = pn.Row(map_component1)
app_layout = pn.Column(image_row, content_bar)
app_layout.show()

Launching server at http://localhost:63913


# District Level Dashboard

In [55]:
#Panel HTML elements can take a 'styles' parameter directly. 
#We are alternatively assigning a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                               css_classes=['title-html-pane'],
                               styles={'color':'white','font-size':'14px','width':'100%',
                                       'background-color':'#6A5638','text-align':'center',
                                       'outline-style':'solid','outline-color':'#30231D',
                                       'outline-width':'thick','font-family':'Times New Roman',
                                       'height':'140px','padding':'10px','border-radius':'10px'})

datalab_img = pn.pane.PNG("Data/Images/datalabimage.png", height=140, width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png", height=140, width=350)

image_row = pn.Row(
    datalab_img,
    title_html_pane,
    txst_img,
    css_classes=['image-box']
)

content_bar = pn.Row(map_component2)
app_layout = pn.Column(image_row, content_bar)
app_layout.show()

Launching server at http://localhost:63914


# Heatmap & Histogram Dashboard

In [56]:
#Panel HTML elements can take a 'styles' parameter directly. 
#We are alternatively assigning a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                               css_classes=['title-html-pane'],
                               styles={'color':'white','font-size':'14px','width':'100%',
                                       'background-color':'#6A5638','text-align':'center',
                                       'outline-style':'solid','outline-color':'#30231D',
                                       'outline-width':'thick','font-family':'Times New Roman',
                                       'height':'140px','padding':'10px','border-radius':'10px'})

datalab_img = pn.pane.PNG("Data/Images/datalabimage.png", height=140, width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png", height=140, width=350)

image_row = pn.Row(
    datalab_img,
    title_html_pane,
    txst_img,
    css_classes=['image-box']
)

content_bar = pn.Row(loss_component1,loss_component2)
app_layout = pn.Column(image_row, content_bar)
app_layout.show()

Launching server at http://localhost:63915


# Definitions

In [81]:
#Panel HTML elements can take a 'styles' parameter directly. 
#We are alternatively assigning a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Learning Loss Recovery after COVID-19 Pandemic in Texas Public Schools</h1>""",
                               css_classes=['title-html-pane'],
                               styles={'color':'white','font-size':'14px','width':'100%',
                                       'background-color':'#6A5638','text-align':'center',
                                       'font-family':'Times New Roman','height':'140px',
                                       'padding':'10px','border-radius':'10px'})

datalab_img = pn.pane.PNG("Data/Images/datalabimage.png", height=140, width=280)
txst_img = pn.pane.PNG("Data/Images/THRClogo.png", height=140, width=350)

image_row = pn.Row(
    datalab_img,
    title_html_pane,
    txst_img,
    css_classes=['image-box']
)

# Manually modifying the HTML/CSS for compatibilities sake
index_pane = pn.pane.HTML(
    """<h1 style='margin-left: 311px;'>Variable Definitions:</h1>""",
    styles={
        'color': 'white',
        'font-size': '20px',
        'width': '100%',
        'margin-left': '300px',
        'outline-style': 'solid',
        'outline-color': '#30231D',
        'outline-width': 'thin',
        'background-color': '#501214',
        'font-family': 'Times New Roman',
        'border': '4px solid black',
        'padding': '5px',
        'border-radius': '5px'
    }
)

table = pn.pane.HTML("""
    <table style="margin: 0 auto; width: 100%; font-size: 16px; text-align: left;">
        <tr>
            <th>Variable</th>
            <th>Reflects the percent difference of...</th>
        </tr>
        <tr>
            <td>STAAR Score in Math</td>
            <td>Math STAAR scores between X and Y years for a district/county.</td>
        </tr>
        <tr>
            <td>STAAR Score in Reading</td>
            <td>Reading STAAR scores between X and Y years for a district/county.</td>
        </tr>
        <tr>
            <td>Labor Force</td>
            <td>Labor force between X and Y years for a county.</td>
        </tr>
        <tr>
            <td>Number of Overall Staff</td>
            <td>Staff between X and Y years for a district/county.</td>
        </tr>
        <tr>
            <td>Number of Students</td>
            <td>Students between X and Y years for a district/county.</td>
        </tr>
        <tr>
            <td>Number of Teachers</td>
            <td>Full-Time Teachers between X and Y years for a district/county.</td>
        </tr>
    </table>
""", styles={
    'width': '100%',
    'font-size': '16px',
    'text-align': 'left',
    'font-family': 'Times New Roman',
    'border': '4px solid black',
    'padding': '10px',
    'border-radius': '10px',
    'margin-left':'300px'
})

content_bar = pn.Column(index_pane, table)
app_layout = pn.Column(image_row, content_bar)
app_layout.show()


Launching server at http://localhost:64951
